# 2D Planar Double Pendulum Golf Swing

## Tucker Knaak - Department of Physics, Creighton University - 2022/2024

#### The golf swing can be modeled as a double pendulum connected to a central hub.  The first pendulum represents the golfer's arms, the second pendulum represents the golf club, and the central hub represents the golfer's chest.  In this model, the golf swing is constrained to the $xy-$plane.  Furthermore, the golf swing has two distinct phases.  In Phase 1, the golfer's wristcock angle is constant, and the equations of motion are known.  In Phase 2, the golfer's arms and the golf club are allowed to accelerate freely, and the equations of motion must be solved numerically.

#### This code is used to solve the equations of motion, plot the relevant data, and animate the golf swing for parameters of the golfer and initial conditions given by the user.

##### $\_\_$init$\_\_$(model_name): The user inputs the name of the model.  This function initializes the relevant data of the class.

##### golfer($r_1$, $m_1$, $r_2$, $m_2$): The user inputs the length and mass of the golfer's arms and the golf club.  This function updates the parameters of the golfer.

##### solve_odes(hub, $L$, $\tau_b$, $\phi_1$, $\phi_2$): The user inputs the hub model, lag parameter, torque of the body, initial arm angle, and initial club angle.  This function numerically solves the equations of motion for the golf swing as stores the relevant data.

##### print_data(): This function prints the time, swing speed, and relevant angles of the system at release, impact, and maximum.

##### plot_data(title): The user inputs the title of the plots.  This function plots the angles, angular velocities, torques, energies, and swing speed versus time.

##### animate_swing(): This function animates the golf swing to be saved as a .gif file.

In [1]:
'''Required Libraries'''
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

'''Required Functions'''
from IPython.display import HTML, display
from numpy import sin, cos

'''Adjust Animation Size'''
matplotlib.rcParams['animation.embed_limit'] = 2**128

In [2]:
class Downswing2D():
    
    '''Internal function to initialize the data for the system'''
    def __init__(self, model_name: str):
        
        '''Model'''
        self.model_name = model_name  #name of the model given by the user
        self.title = ''               #title for the plots of relevant data and animations of the swing given by the user
        
        '''Parameters of the Golfer'''
        self.L = 0               #lag parameter
        self.tau_b = 0           #torque of the body [N * m]
        self.tau_h_points = []   #torque of the hands [N * m]
        self.r1, self.m1 = 0, 0  #length [m] and mass [kg] of the golfer's arms
        self.r2, self.m2 = 0, 0  #length [m] and mass [kg] of the golf club
        
        '''Moments of Inertia'''
        self.A = 0  #(m1 + m2) * r1^2 [kg * m^2]
        self.B = 0  #m2 * r2^2 [kg * m^2]
        self.C = 0  #m2 * r1 * r2 [kg * m^2]
        self.D = 0  #(m1 + m2) * r1 [kg * m]
        self.E = 0  #m2 * r2 [kg * m]
        
        '''Times'''
        self.z_points = []  #parameterized time [sqrt{N * m} * s]
        self.t_points = []  #time [s]
        self.t_release = 0  #time when the golfer's hands release [s]
        self.t_impact = 0   #time when the golf club impacts the golf ball [s]
        self.t_ss_maxx = 0  #time when the swing speed is a maximum [s]
        
        '''Hub'''
        self.hub_model = ''     #constant or accelerating hub given by the user
        self.hub_x_points = []  #x-coordinate of the golfer's chest [m]
        self.hub_y_points = []  #y-coordinate of the golfer's chest [m]
        
        '''Arms'''
        self.x1_points = []        #x-coordinate of the golfer's hands [m]
        self.y1_points = []        #y-coordinate of the golfer's hands [m]
        self.phi1_points = []      #angle of the golfer's arms ccw from the y-axis in the xy-plane [deg]
        self.phi1dot_points = []   #angular velocity of phi1 [deg / s]
        self.phi1dotz_points = []  #parameterized angular velocity of phi1 [deg / sqrt{N * m} * s]
        self.phi1_crit = 0         #phi1 when the golfer's hands release [deg]
        
        '''Club'''
        self.x2_points = []        #x-coordinate of the golf club [m]
        self.y2_points = []        #y-coordinate of the golf club [m]
        self.phi3_points = []      #angle between the golf club & golfer's arms ccw from the y-axis in the xy-plane [deg]
        self.phi3dot_points = []   #angular velocity of phi3 [deg / s]
        self.phi3dotz_points = []  #parameterized angular velocity of phi3 [deg / sqrt{N * m} * s]
        self.aoa = 0               #direction of the golf club in the xy-plane at impact [deg]
        self.ball_x = 0            #x-coordinate of the golf clubhead at impact [m]
        self.ball_y = 0            #y-coordinate of the golf clubhead at impact [m]
        
        '''Energies'''
        self.ke_points = []      #kinetic energy produced during the downswing [J]
        self.pe_points = []      #potential energy produced during the downswing [J]  
        self.totale_points = []  #total energy produced during the downswing [J]
        
        '''Swing Speeds'''
        self.ss_points = []  #speed of the golf clubhead during the downswing [m / s]
        self.ss_release = 0  #speed of the golf clubhead at release [m / s]
        self.ss_impact = 0   #speed of the golf clubhead at impact [m / s]
        self.ss_max = 0      #maximum speed of the golf clubhead during the downswing [m / s]
        
        '''Algorithm'''
        self.dt = 0.00175           #time step each iteration [s]
        self.dz = 0                 #parameterized time step each iteration [sqrt{N * m} * s]
        self.count = 0              #counter used to ensure only one impact location
        self.num_steps_release = 0  #number of iterations at release
        self.num_steps_impact = 0   #number of iterations at impact
        self.num_steps_ss_max = 0   #number of iterations at the maximum swing speed
        
        '''Yoshida Coefficients'''
        self.w0 = -np.cbrt(2) / (2 - np.cbrt(2))
        self.w1 = 1 / (2 - np.cbrt(2))
        self.c1 = self.w1 / 2
        self.c2 = (self.w0 + self.w1) / 2
        self.c_points = np.array([self.c1, self.c2, self.c2, self.c1], dtype = float)  #position time steps
        self.d_points = np.array([self.w1, self.w0, self.w1], dtype = float)           #velocity time steps
        
        
    '''Internal function to return the x & y position, velocity, and acceleration of a constant hub model'''
    def _constant_hub(self, z: float):
        h, hdotz, hddotz = 0, 0, 0
        v, vdotz, vddotz = 0, 0, 0
        return h, hdotz, hddotz, v, vdotz, vddotz
        
        
    '''Internal function to return the x & y position, velocity, and acceleration of an accelerating hub model'''
    def _accelerating_hub(self, z: float):
        h = (0.9789 * z**2 / self.tau_b) + (0.1628 * z / np.sqrt(self.tau_b)) - 0.1270
        hdotz = (1.9578 * z / self.tau_b) + (0.1628 / np.sqrt(self.tau_b))
        hddotz = (1.9578 / self.tau_b)
        v = (1.8355 * z**2 / self.tau_b) - (0.7584 * z / np.sqrt(self.tau_b)) + 0.0170
        vdotz = (3.6710 * z / self.tau_b) - (0.7584 / np.sqrt(self.tau_b))
        vddotz = 3.6710 / self.tau_b
        return h, hdotz, hddotz, v, vdotz, vddotz
        
        
    '''Internal function to calculate the xy-coordinates of the golfer"s hands and the golf clubhead'''
    def _coordinates(self, phi1: float, phi3: float, h: float, v: float):
        phi2 = phi1 + phi3
        x1 = -self.r1 * sin(phi1) + h
        y1 = self.r1 * cos(phi1) + v
        x2 = x1 - self.r2 * sin(phi2)
        y2 = y1 + self.r2 * cos(phi2)
        return x1, y1, x2, y2
        
        
    '''Internal function to calculate the torque of the hands during Phase 1'''
    def _torque_hands(self, phi1: float, hddotz: float, vddotz: float):
        tau_h = self.tau_b * ((((self.B * (self.L + 1)) - 2 * phi1 * self.C) * (1 + self.D * hddotz - self.E * vddotz) /
                             (self.A + self.B)) + self.E * vddotz)
        return tau_h
        
        
    '''Internal function to calculate the speed of the golf clubhead'''
    def _swing_speed(self, phi1: float, phi3: float, phi1dot: float, phi3dot: float, hdot: float, vdot: float):
        phi2, phi2dot = phi1 + phi3, phi1dot + phi3dot
        x2dot = -self.r1 * phi1dot * cos(phi1) - self.r2 * phi2dot * cos(phi2) + hdot
        y2dot = -self.r1 * phi1dot * sin(phi1) - self.r2 * phi2dot * sin(phi2) + vdot
        return np.sqrt(x2dot**2 + y2dot**2)
        
        
    '''Internal function to calculate the kinetic energy produced by the system'''
    def _kinetic_energy(self, phi1: float, phi3: float, phi1dot: float, phi3dot: float, hdot: float, vdot: float):
        phi2, phi2dot = phi1 + phi3, phi1dot + phi3dot
        T1 = 0.5 * self.m1 * ((self.r1 * phi1dot)**2 + hdot**2 + vdot**2
             - (2 * self.r1 * phi1dot * (hdot * cos(phi1) + vdot * sin(phi1))))
        T2 = 0.5 * self.m2 * ((self.r1 * phi1dot)**2 + (self.r2 * phi2dot)**2 + hdot**2 + vdot**2
             - (2 * self.r1 * phi1dot * (hdot * cos(phi1) + vdot * sin(phi1)))
             - (2 * self.r2 * phi2dot * (hdot * cos(phi2) + vdot * sin(phi2)))
             + (2 * self.r1 * self.r2 * phi1dot * phi2dot * cos(phi2 - phi1)))
        return T1 + T2
        
        
    '''Internal function to calculate the potential energy produced by the system'''
    def _potential_energy(self, phi1: float, phi3: float, tau_h: float):
        phi2 = phi1 + phi3
        V1 = -self.tau_b * phi1
        V2 = -tau_h * (phi2 - phi1)
        return V1 + V2
        
        
    '''Internal function to calculate the phi1 & phi3 accelerations during Phase 2'''
    def _acceleration(self, pos: list, vel: list, hddotz: float, vddotz: float):
        phi1, phi3 = pos[0], pos[1]
        phi1dotz, phi3dotz = 0.9 * vel[0], vel[1]
        phi1ddotz = 0.9 * ((self.E * self.C * cos(phi3) * (hddotz * cos(phi1 + phi3) + vddotz * sin(phi1 + phi3)))
                    - (self.B * self.D * (hddotz * cos(phi1) + vddotz * sin(phi1))) - self.B
                    - (self.B * self.C * sin(phi3) * (phi1dotz + phi3dotz)**2)
                    - (sin(phi3) * cos(phi3) * (self.C * phi1dotz)**2)) / ((self.C * cos(phi3))**2 - (self.A * self.B))
        phi3ddotz = (1 + (self.D * (hddotz * cos(phi1) + vddotz * sin(phi1)))
                    + (self.E * (hddotz * cos(phi1 + phi3) + vddotz * sin(phi1 + phi3)))
                    - ((self.A + self.B + 2 * self.C * cos(phi3)) * phi1ddotz)
                    + (self.C * ((phi1dotz + phi3dotz)**2 - phi1dotz**2) * sin(phi3))) / (self.B + self.C * cos(phi3))
        return np.array([phi1ddotz, phi3ddotz], dtype = float)
        
        
    '''Internal function to iterate the next time step with the Yoshida Predictor-Corrector algorithm during Phase 2'''
    def _YoshidaPC(self, pos: list, vel_c: list, hddotz: float, vddotz: float):
        for c, d in zip(self.c_points, self.d_points):
            pos += c * vel_c * self.dz
            vel_p = vel_c + d * self._acceleration(pos, vel_c, hddotz, vddotz) * self.dz
            vel_c += d * self._acceleration(pos, vel_p, hddotz, vddotz) * self.dz
        pos += self.c_points[-1] * vel_c * self.dz
        return pos, vel_c
        
        
    '''Internal function to parameterize the relevant data from z to t'''
    def _parameterize(self, z: float, phi1dotz: float, phi3dotz: float, hdotz: float, hddotz: float,
                      vdotz: float, vddotz: float):
        t = round(z / np.sqrt(self.tau_b), 4)
        phi1dot = phi1dotz * np.sqrt(self.tau_b)
        phi3dot = phi3dotz * np.sqrt(self.tau_b)
        hdot = hdotz * np.sqrt(self.tau_b)
        hddot = hddotz * self.tau_b
        vdot = vdotz * np.sqrt(self.tau_b)
        vddot = vddotz * self.tau_b
        return t, phi1dot, phi3dot, hdot, hddot, vdot, vddot
        
        
    '''Internal function to calculate the relevant data of the system'''
    def _calculate(self, phi1: float, phi3: float, phi1dot: float, phi3dot: float, h: float, hdot: float,
                   v: float, vdot: float, tau_h: float):
        x1, y1, x2, y2 = self._coordinates(phi1, phi3, h, v)
        ss = self._swing_speed(phi1, phi3, phi1dot, phi3dot, hdot, vdot)
        ke = self._kinetic_energy(phi1, phi3, phi1dot, phi3dot, hdot, vdot)
        pe = self._potential_energy(phi1, phi3, tau_h)
        totale = ke + pe
        return x1, y1, x2, y2, ss, ke, pe, totale
        
        
    '''Internal function to append the relevant data of the system to their respective lists'''
    def _append(self, z: float, t: float, h: float, v: float, x1: float, y1: float, x2: float, y2: float,
                phi1: float, phi3: float, phi1dot: float, phi1dotz: float, phi3dot: float, phi3dotz: float,
                ss: float, ke: float, pe: float, totale: float, tau_h: float):
        self.z_points.append(z)
        self.t_points.append(t)
        self.hub_x_points.append(h)
        self.hub_y_points.append(v)
        self.x1_points.append(x1)
        self.y1_points.append(y1)
        self.x2_points.append(x2)
        self.y2_points.append(y2)
        self.phi1_points.append(phi1)
        self.phi3_points.append(phi3)
        self.phi1dot_points.append(phi1dot)
        self.phi1dotz_points.append(phi1dotz)
        self.phi3dot_points.append(phi3dot)
        self.phi3dotz_points.append(phi3dotz)
        self.ss_points.append(ss)
        self.ke_points.append(ke)
        self.pe_points.append(pe)
        self.totale_points.append(totale)
        self.tau_h_points.append(tau_h)
        
        
    '''Internal function to parameterize, calculate, and append the relevant data of the system'''
    def _step(self, z: float, phi1: float, phi3: float, phi1dotz: float, phi3dotz: float,
              h: float, hdotz: float, hddotz: float, v: float, vdotz: float, vddotz: float, tau_h: float):
        t, phi1dot, phi3dot, hdot, hddot, vdot, vddot = self._parameterize(z, phi1dotz, phi3dotz, hdotz, hddotz,
                                                                           vdotz, vddotz)
        x1, y1, x2, y2, ss, ke, pe, totale = self._calculate(phi1, phi3, phi1dot, phi3dot, h, hdot, v, vdot, tau_h)
        self._append(z, t, h, v, x1, y1, x2, y2, phi1, phi3, phi1dot, phi1dotz, phi3dot, phi3dotz,
                     ss, ke, pe, totale, tau_h)
        
        
    '''Internal function to update the frame of the animation of the downswing'''
    def _update_swing(self, i, fig, ax, swing, hub, hands, clubhead):
        
        '''Set color to red at time of release'''
        if i in [step for step in range(self.num_steps_release - 3, self.num_steps_release + 4)]:
            swing_color, hub_color, hands_color, clubhead_color = 'red', 'red', 'red', 'red'
        else:
            swing_color, hub_color, hands_color, clubhead_color = 'black', 'black', 'bisque', 'darkgray'
            
        '''Update frame'''
        swing.set_data([self.hub_x_points[i], self.x1_points[i], self.x2_points[i]],
                       [self.hub_y_points[i], self.y1_points[i], self.y2_points[i]])
        swing.set_color(f'{swing_color}')
        hub.set_data([self.hub_x_points[i]], [self.hub_y_points[i]])
        hub.set_color(f'{hub_color}')
        hands.set_data([self.x1_points[i]], [self.y1_points[i]])
        hands.set_color(f'{hands_color}')
        clubhead.set_data([self.x2_points[i]], [self.y2_points[i]])
        clubhead.set_color(f'{clubhead_color}')
        
        '''Return Artist elements'''
        return swing, hub, hands, clubhead
        
        
    '''Internal function to find the frames of the still-shot image of the positions of the downswing'''
    def _find_frames(self):
        
        '''Include start position'''
        frames = [0]
        prev_angle = self.phi1_points[0]
        
        '''Add new frame when phi1 has increased by 10 degrees'''
        for i in range(1, len(self.phi1_points)):
            if self.phi1_points[i] <= 270:
                pass
            else:
                break
            if self.phi1_points[i] - prev_angle >= 10:
                frames.append(i)
                prev_angle = self.phi1_points[i]
                
        '''Include release and impact'''
        frames.append(self.num_steps_release) if self.num_steps_release not in frames else None
        frames.append(self.num_steps_impact) if self.num_steps_impact not in frames else None
        
        '''Check if release and impact are within 2 frames of other frames'''
        for frame in frames:
            if frame != self.num_steps_release and abs(frame - self.num_steps_release) <= 2:
                frames.remove(frame)
            elif frame != self.num_steps_impact and abs(frame - self.num_steps_impact) <= 2:
                frames.remove(frame)
                
        '''Return frames'''
        return sorted(frames)
        
        
    '''Function called by the user to set the parameters of the golfer and calculate the moments of inertia'''
    def golfer(self, r1: float, m1: float, r2: float, m2: float):
        self.r1 = r1
        self.m1 = m1
        self.r2 = r2
        self.m2 = m2
        self.A = (m1 + m2) * r1**2
        self.B = m2 * r2**2
        self.C = m2 * r1 * r2
        self.D = (m1 + m2) * r1
        self.E = m2 * r2
        
        
    '''Function called by the user to solve the system of differential equations of the downswing'''
    def solve_odes(self, hub_model: str, L: float, tau_b: float, phi1_0: float, phi2_0: float):
        
        '''Set parameters and initial conditions'''
        self.L = L              #lag parameter
        self.tau_b = tau_b      #torque of the body [N * m]
        phi1 = phi1_0           #arm angle phi1 [deg]
        phi3 = phi2_0 - phi1_0  #wristcock angle phi3 [deg]
        phi1dotz = 0.0          #angular velocity of phi1 [deg / sqrt{N * m} * s]
        phi3dotz = 0.0          #angular velocity of phi3 [deg / sqrt{N * m} * s]
        
        '''Set initial parameterized time'''
        z = 0.0                                  #parameterized time [sqrt{N * m} * s]
        self.dz = np.sqrt(self.tau_b) * self.dt  #parameterized time step each iteration [sqrt{N * m} * s]
        
        '''Set hub model'''
        if hub_model.lower() == 'constant' or hub_model.lower() == 'c':
            hub_func = lambda z: self._constant_hub(z)
            self.hub_model = 'Constant Hub'
        elif hub_model.lower() == 'accelerating' or hub_model.lower() == 'a':
            hub_func = lambda z: self._accelerating_hub(z)
            self.hub_model = 'Accelerating Hub'
        else:
            raise ValueError('Invalid choice for the hub model.  Please enter "constant" or "accelerating"!')
        h, hdotz, hddotz, v, vdotz, vddotz = hub_func(z)  #calculate initial hub variables
        
        '''Calculate constant acceleration of phi1 during Phase 1'''
        phi1ddotz = (1 + self.D * hddotz - self.E * vddotz) / (self.A + self.B)
        
        '''Calculate initial torque of the hands'''
        tau_h = self._torque_hands(phi1, hddotz, vddotz)
        
        '''Parameterize from z to t / calculate relevant data / append relevant data'''
        self._step(z, phi1, phi3, phi1dotz, phi3dotz, h, hdotz, hddotz, v, vdotz, vddotz, tau_h)
        
        '''Phase 1 --> constant wristcock angle until tau_b = 0 and the golfer"s hands release'''
        while tau_h > 0:
            
            '''Iterate next z step'''
            z += self.dz
            
            '''Update hub data / Update phi1, phi1dotz from known solution'''
            h, hdotz, hddotz, v, vdotz, vddotz = hub_func(z)
            phi1 = phi1_0 + 0.5 * phi1ddotz * z**2
            phi1dotz = phi1ddotz * z
            tau_h = self._torque_hands(phi1, hddotz, vddotz)
            if tau_h <= 0:
                tau_h = 0
            
            '''Parameterize from z to t / calculate relevant data / append relevant data'''
            self._step(z, phi1, phi3, phi1dotz, phi3dotz, h, hdotz, hddotz, v, vdotz, vddotz, tau_h)
            
        '''Calculate data at release'''
        self.t_release = self.t_points[-1]
        self.ss_release = self.ss_points[-1]
        self.num_steps_release = len(self.t_points) - 1
        self.phi1_crit = round(np.degrees(self.phi1_points[-1]), 1)
        
        '''Set initial conditions for Phase 2'''
        pos = np.array([self.phi1_points[-1], self.phi3_points[-1]], dtype = float)
        vel_c = np.array([self.phi1dotz_points[-1], self.phi3dotz_points[-1]], dtype = float)
        
        '''Phase 2 --> golfer"s hands release and phi1 & phi3 are allowed to accelerate freely'''
        while np.degrees(phi1) < 345:
            
            '''Iterate next z step'''
            z += self.dz
            
            '''Update hub data / Update phi1, phi3, phi1dotz, phi3dotz from Yoshida Predictor-Corrector Algorithm'''
            h, hdotz, hddotz, v, vdotz, vddotz = hub_func(z)
            pos, vel_c = self._YoshidaPC(pos, vel_c, hddotz, vddotz)
            phi1, phi3 = pos
            phi1dotz, phi3dotz = vel_c
            
            '''Parameterize from z to t / calculate relevant data / append relevant data'''
            self._step(z, phi1, phi3, phi1dotz, phi3dotz, h, hdotz, hddotz, v, vdotz, vddotz, tau_h)
            
            '''Calculate data at impact'''
            if self.x2_points[-1] >= 0.285 and self.x2_points[-1] <= 0.385 and self.x1_points[-1] >= 0:
                if self.count == 0:
                    self.t_impact = self.t_points[-1]
                    self.ss_impact = self.ss_points[-1]
                    self.num_steps_impact = len(self.t_points) - 1
                    self.ball_x, self.ball_y = self.x2_points[-1], self.y2_points[-1]
                    delta_x = self.x2_points[-1] - self.x2_points[-2]
                    delta_y = self.y2_points[-1] - self.y2_points[-2]
                    self.aoa = round(np.degrees(np.arctan(delta_y / delta_x)), 1)
                    self.count += 1
        
        '''Calculate maximum swing speed data'''
        self.ss_max = max(self.ss_points)
        self.num_steps_ss_max = self.ss_points.index(self.ss_max)
        self.t_ss_max = self.t_points[self.num_steps_ss_max]
        
        '''Change units from radians to degrees'''
        self.phi1_points = np.degrees(self.phi1_points)
        self.phi3_points = np.degrees(self.phi3_points)
        self.phi1dot_points = np.degrees(self.phi1dot_points)
        self.phi3dot_points = np.degrees(self.phi3dot_points)
        
        
    '''Function called by the user to print the relevant data at important points during the downswing'''
    def print_data(self):
        print(f'Model: {self.model_name}')
        print(f'Release: {self.ss_release:.2f}m/s ({self.t_release}s)')
        print(f'Impact: {self.ss_impact:.2f}m/s ({self.t_impact}s)')
        print(f'Max: {self.ss_max:.2f}m/s ({self.t_ss_max}s)')
        print(f'Critical Arm Angle: {self.phi1_crit}\u00B0')
        print(f'Angle of Attack: {self.aoa}\u00B0')
        print(f'Ball Position: ({self.ball_x:.2f}m, {self.ball_y:.2f}m)')
        
        
    '''Function called by the user to plot the relevant data versus time during the downswing'''
    def plot_data(self, title: str):
        
        '''Set title'''
        self.title = title
        
        '''Create colormap for plots'''
        colors = plt.cm.magma(np.linspace(0.9, 0.1, 11))
        
        '''Set num_steps_release and num_steps_impact'''
        r, i = self.num_steps_release, self.num_steps_impact
        
        '''Create figure'''
        fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(3, 3,
                figsize = (21, 15), gridspec_kw = {'width_ratios': [1, 1, 1], 'height_ratios': [1, 1, 1]})
        fig.tight_layout(pad = 4)
        
        '''First figure --> tau_h(t)'''
        ax1.set_xlabel('$t$ [s]', fontsize = 12)
        ax1.set_ylabel('$\u03C4_h$ [N\u00B7m]')
        ax1.set_title(f'Torque of the Hands - 2D {self.hub_model} ({title})')
        ax1.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax1.plot(self.t_points, self.tau_h_points, linestyle = 'solid', color = colors[9])
        ax1.scatter([self.t_points[r], self.t_points[i]], [self.tau_h_points[r], self.tau_h_points[i]], c = 'black')
        
        '''Second figure --> phi1(t)'''
        ax2.set_xlabel('$t$ [s]', fontsize = 12)
        ax2.set_ylabel('$\u03C6_1$ [\u00B0]')
        ax2.set_title(f'Arm Angle - 2D {self.hub_model} ({title})')
        ax2.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax2.plot(self.t_points, self.phi1_points, linestyle = 'solid', color = colors[1])
        ax2.scatter([self.t_points[r], self.t_points[i]], [self.phi1_points[r], self.phi1_points[i]],
                     c = 'black')
        
        '''Third figure --> phi3(t)'''
        ax3.set_xlabel('$t$ [s]', fontsize = 12)
        ax3.set_ylabel('$\u03C6_3$ [\u00B0]')
        ax3.set_title(f'Wristcock Angle - 2D {self.hub_model} ({title})')
        ax3.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax3.plot(self.t_points, self.phi3_points, linestyle = 'solid', color = colors[2])
        ax3.scatter([self.t_points[r], self.t_points[i]], [self.phi3_points[r], self.phi3_points[i]],
                     c = 'black')
        
        '''Fourth figure --> ss(t)'''
        ax4.set_xlabel('$t$ [s]', fontsize = 12)
        ax4.set_ylabel('Speed [m/s]')
        ax4.set_title(f'Swing Speed - 2D {self.hub_model} ({title})')
        ax4.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax4.plot(self.t_points, self.ss_points, linestyle = 'solid', color = colors[10])
        ax4.scatter([self.t_points[r], self.t_points[i]], [self.ss_points[r], self.ss_points[i]], c = 'black')
        
        '''Fifth figure --> phi1dot(t)'''
        ax5.set_xlabel('$t$ [s]', fontsize = 12)
        ax5.set_ylabel('$\dot{\u03C6}_1$ [\u00B0/s]')
        ax5.set_title(f'Arm Angular Velocity - 2D {self.hub_model} ({title})')
        ax5.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax5.plot(self.t_points, self.phi1dot_points, linestyle = 'solid', color = colors[4])
        ax5.scatter([self.t_points[r], self.t_points[i]], [self.phi1dot_points[r], self.phi1dot_points[i]],
                     c = 'black')
        
        '''Sixth figure --> phi3dot(t)'''
        ax6.set_xlabel('$t$ [s]', fontsize = 12)
        ax6.set_ylabel('$\dot{\u03C6}_3$ [\u00B0/s]')
        ax6.set_title(f'Wristcock Angular Velocity - 2D {self.hub_model} ({title})')
        ax6.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax6.plot(self.t_points, self.phi3dot_points, linestyle = 'solid', color = colors[5])
        ax6.scatter([self.t_points[r], self.t_points[i]], [self.phi3dot_points[r], self.phi3dot_points[i]],
                     c = 'black')
        
        '''Seventh figure --> ke(t)'''
        ax7.set_xlabel('$t$ [s]', fontsize = 12)
        ax7.set_ylabel('KE [J]')
        ax7.set_title(f'Kinetic Energy - 2D {self.hub_model} ({title})')
        ax7.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax7.plot(self.t_points, self.ke_points, linestyle = 'solid', color = colors[6])
        ax7.scatter([self.t_points[r], self.t_points[i]], [self.ke_points[r], self.ke_points[i]], c = 'black')
        
        '''Eighth figure --> pe(t)'''
        ax8.set_xlabel('$t$ [s]', fontsize = 12)
        ax8.set_ylabel('PE [J]')
        ax8.set_title(f'Potential Energy - 2D {self.hub_model} ({title})')
        ax8.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax8.plot(self.t_points, self.pe_points, linestyle = 'solid', color = colors[7])
        ax8.scatter([self.t_points[r], self.t_points[i]], [self.pe_points[r], self.pe_points[i]], c = 'black')
        
        '''Ninth figure --> totale(t)'''
        ax9.set_xlabel('$t$ [s]', fontsize = 12)
        ax9.set_ylabel('E [J]')
        ax9.set_title(f'Total Energy - 2D {self.hub_model} ({title})')
        ax9.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        ax9.plot(self.t_points, self.totale_points, linestyle = 'solid', color = colors[8])
        ax9.scatter([self.t_points[r], self.t_points[i]], [self.totale_points[r], self.totale_points[i]], c = 'black')
        
        
    '''Function called by the user to animate the downswing'''
    def animate_swing(self, save_fig = True):
        
        '''Create figure'''
        fig, ax = plt.subplots(1, 1, figsize = (6, 6))
        fig.tight_layout(pad = 5)
        ax.set_aspect('equal', 'box')
        ax.set_xlim(-2.25, 2.25)
        ax.set_ylim(-2.25, 2.25)
        ax.set_xlabel('$x$ [m]')
        ax.set_ylabel('$y$ [m]')
        ax.set_title(f'2D {self.hub_model} ({self.title})')
        ax.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        
        '''Initialize downswing'''
        swing, = ax.plot([], [], linestyle = 'solid', linewidth = 2, color = 'black', zorder = 1)
        hub, = ax.plot([], [], linestyle = 'None', marker = 'o', markersize = 2, zorder = 10)
        hands, = ax.plot([], [], linestyle = 'None', marker = 'o', markersize = 8, zorder = 10)
        clubhead, = ax.plot([], [], linestyle = 'None', marker = 'o', markersize = 8, zorder = 10)
        ax.plot([], [], 'r-', markersize = 8, label = 'Release')
        ax.plot([self.ball_x], [self.ball_y], 'ko', markersize = 8, markeredgewidth = 1.5, fillstyle = 'none')
        legend = ax.legend(loc = 'upper left', fancybox = False)
        frame = legend.get_frame()
        frame.set_facecolor('white')
        frame.set_edgecolor('black')
        frame.set_linewidth(1)
        
        '''Create animation'''
        frames = [i for i in range(0, len(self.t_points), 2)]
        ani = animation.FuncAnimation(fig, self._update_swing, interval = 25, frames = frames,
                                      fargs = (fig, ax, swing, hub, hands, clubhead), blit = True, repeat = False)
        html = HTML(ani.to_jshtml())
        display(html)
        plt.close()
        
        '''Save animation'''
        if save_fig:
            f = r'c:/Users/Tucker Knaak/Downloads/{}.gif'.format(self.model_name)
            fig.tight_layout(pad = 1.25)
            writergif = animation.PillowWriter(fps = 50)
            ani.save(f, writer = writergif)
        
        
        
    '''Function called by the user to create a still-shot image of the positions of the downswing'''
    def still_shot(self, save_fig = True):
        
        '''Create figure'''
        fig, ax = plt.subplots(1, 1, figsize = (6, 6))
        fig.tight_layout(pad = 1.25)
        ax.set_aspect('equal', 'box')
        ax.set_xlim(-2.25, 2.25)
        ax.set_ylim(-2.25, 2.25)
        ax.set_xlabel('$x$ [m]')
        ax.set_ylabel('$y$ [m]')
        ax.set_title(f'2D {self.hub_model} ({self.title})')
        ax.grid(True, linestyle = 'dashed', color = 'darkgray', alpha = 0.25)
        
        '''Plot downswing'''
        frames = self._find_frames()
        for i in frames:
            if i == self.num_steps_release:
                swing_color = 'red'
            else:
                swing_color = 'black'
            ax.plot([self.hub_x_points[i], self.x1_points[i], self.x2_points[i]],
                    [self.hub_y_points[i], self.y1_points[i], self.y2_points[i]],
                    linestyle = 'solid', linewidth = 2, color = f'{swing_color}')
        ax.plot([], [], 'r-', label = 'Release')
        ax.plot([self.ball_x], [self.ball_y], 'ko', markersize = 8, markeredgewidth = 1.5, fillstyle = 'none')
        legend = ax.legend(loc = 'upper left', fancybox = False)
        frame = legend.get_frame()
        frame.set_facecolor('white')
        frame.set_edgecolor('black')
        frame.set_linewidth(1)
        
        '''Save figure'''
        if save_fig:
            fig.savefig('c:/Users/Tucker Knaak/Downloads/{}.png'.format(self.model_name), bbox_inches = 'tight')